In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [2]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('CUDA version:', torch.version.cuda)
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce GTX 960
CUDA version: 11.7
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [4]:
import numpy as np
import PIL
from PIL import Image
import datasets
import evaluate
import torch
import json
import codecs
import os
from os import sys

from transformers import AutoImageProcessor
from transformers import SegformerImageProcessor

from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer, EarlyStoppingCallback

c:\Users\Kate\AppData\Local\miniconda3\envs\master\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
module_path = os.path.abspath(os.path.join('./src'))
if module_path not in sys.path:
    sys.path.append(module_path)
from data_prepossessing import create_datasets_for_plants, get_labels
from constants import *

In [6]:
checkpoint = "nvidia/mit-b0"
image_processor = SegformerImageProcessor.from_pretrained(checkpoint)
image_processor

c:\Users\Kate\AppData\Local\miniconda3\envs\master\Lib\site-packages\transformers\models\segformer\image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


SegformerImageProcessor {
  "do_normalize": true,
  "do_reduce_labels": false,
  "do_rescale": true,
  "do_resize": true,
  "feature_extractor_type": "SegformerFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "SegformerImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 512,
    "width": 512
  }
}

In [7]:
def train_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels, return_tensors="pt")
    return inputs

In [8]:
metric = evaluate.load("mean_iou")

In [9]:
def compute_metrics(num_labels, eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = torch.nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

In [10]:
# TODO try to remove eval_accumulation_steps
training_args = TrainingArguments(
    output_dir="segformer-b0-scene-parse-150",
    learning_rate=6e-5,
    num_train_epochs=25,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    eval_steps=100,
    logging_steps=1,
    # eval_accumulation_steps=5,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    seed=seed,
)

In [11]:
def initialize_trainer(model, num_labels, train_ds, test_ds) :
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=lambda eval_pred: compute_metrics(num_labels, eval_pred),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    return trainer

In [12]:
def train_model_of_type_for_crop(model_type, crop):
    model_plant_names = [crop] + weed_plants
    train_ds, val_ds, test_ds = create_datasets_for_plants(model_plant_names, model_type, crop)

    print("Training subset number of images: " + str(train_ds.num_rows))
    print("Validation subset number of images: " + str(val_ds.num_rows))
    print("Test subset number of images: " + str(test_ds.num_rows))

    train_ds.set_transform(train_transforms)
    val_ds.set_transform(train_transforms)
    test_ds.set_transform(train_transforms)

    id2label, label2id = get_labels(crop, model_type)

    print('Number of classes:', len(id2label))
    print('id2label:', id2label)
    print('label2id:', label2id)

    model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)
    trainer = initialize_trainer(model, len(id2label), train_ds, val_ds)
    trainer.train()

    # Save the trained model, so that it can be used for inference later.
    # Save the log history, so that it can be used for plotting later.
    trainer.save_model('models/' + model_type + '/' + crop)
    with open('models/' + model_type + '/' + crop + '/log_history.json', 'w') as file:
        log_history = trainer.state.log_history
        json.dump(log_history, file)

    test_metric = trainer.evaluate(test_ds)
    test_metric

    with open('models/' + model_type + '/' + crop + '/test_metric.json', 'w') as file:
        json.dump(test_metric, file)

In [13]:
train_model_of_type_for_crop("multiclass", "broad_bean")

['img_00173.png', 'img_00174.png', 'img_00175.png', 'img_00176.png', 'img_00177.png', 'img_00178.png', 'img_00672.png', 'img_00673.png', 'img_00674.png', 'img_00675.png', 'img_00676.png', 'img_00677.png', 'img_00678.png', 'img_00679.png', 'img_00680.png', 'img_00681.png', 'img_00682.png', 'img_00683.png', 'img_00684.png', 'img_00882.png', 'img_00883.png', 'img_00884.png', 'img_00885.png', 'img_00886.png', 'img_00887.png', 'img_00938.png', 'img_00980.png', 'img_00981.png', 'img_00982.png', 'img_00983.png', 'img_00984.png', 'img_00985.png', 'img_00986.png', 'img_00987.png', 'img_00988.png', 'img_00989.png', 'img_01070.png', 'img_01071.png', 'img_01072.png', 'img_01073.png', 'img_01074.png', 'img_01075.png', 'img_01076.png', 'img_01077.png', 'img_01078.png', 'img_01079.png', 'img_01219.png', 'img_01220.png', 'img_01221.png', 'img_01222.png', 'img_01223.png', 'img_01224.png', 'img_01225.png', 'img_01226.png', 'img_01227.png', 'img_01228.png', 'img_01279.png', 'img_01280.png', 'img_01281.pn

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.running_var', 'decode_head.batch_norm.running_mean', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.classifier.bias', 'decode_head.linear_c.0.proj.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.classifier.weight', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_fuse.weight', 'decode_head.batch_norm.weight', 'decode_head.linear_c.1.proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Kate\AppData\Local\miniconda3\envs\master\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and wi

{'loss': 2.3566, 'learning_rate': 5.9959322033898307e-05, 'epoch': 0.02}


  0%|          | 2/1475 [00:14<2:51:23,  6.98s/it]

{'loss': 2.3155, 'learning_rate': 5.991864406779661e-05, 'epoch': 0.03}


...

  2%|▏         | 36/1475 [03:17<2:09:04,  5.38s/it]

{'loss': 1.6735, 'learning_rate': 5.853559322033899e-05, 'epoch': 0.61}


  3%|▎         | 37/1475 [03:23<2:09:02,  5.38s/it]

{'loss': 1.6152, 'learning_rate': 5.849491525423729e-05, 'epoch': 0.63}


KeyboardInterrupt: 

In [ ]:
# import subprocess
# from typing import NoReturn

# def shutdown_windows() -> NoReturn:
#     subprocess.run(["shutdown", "/s", "/t", "0"])

# shutdown_windows()